In [8]:
import pandas as pd
import joblib
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

def carregar_dados_treino(caminho_x, caminho_y):
    """Carrega os dados que já foram escalonados no pré-processamento."""
    X_train = pd.read_csv(caminho_x)
    y_train = pd.read_csv(caminho_y).values.ravel()
    return X_train, y_train

def definir_modelos(random_state=42):
    """
    Retorna um dicionário com os modelos configurados.
    Como os dados já chegam escalonados, não precisamos de Pipelines aqui.
    """
    return {
        "RandomForest": RandomForestClassifier(
            n_estimators=100, 
            max_depth=10, 
            random_state=random_state, 
            class_weight="balanced"
        ),
        "XGBoost": XGBClassifier(
            n_estimators=100, 
            random_state=random_state, 
            use_label_encoder=False, 
            eval_metric='mlogloss'
        ),
        "LogisticRegression": LogisticRegression(
            class_weight='balanced', 
            max_iter=1000, 
            random_state=random_state
        ),
        "SVM": SVC(
            kernel='rbf', 
            probability=True, 
            class_weight='balanced', 
            random_state=random_state
        )
    }

def salvar_modelo(modelo, nome_modelo):
    """Salva o modelo treinado na pasta de modelos."""
    caminho_saida = f"../src/models/{nome_modelo}.pkl"
    os.makedirs(os.path.dirname(caminho_saida), exist_ok=True)
    joblib.dump(modelo, caminho_saida)
    print(f"Modelo {nome_modelo} salvo em: {caminho_saida}")

def executar_pipeline_treinamento():
    """Fluxo principal de treinamento multimodelo."""
    # Caminhos para os dados processados (já escalonados com RobustScaler)
    X_TRAIN_PATH = "../data/processed/X_train.csv"
    Y_TRAIN_PATH = "../data/processed/y_train.csv"

    # 1. Carga
    X_train, y_train = carregar_dados_treino(X_TRAIN_PATH, Y_TRAIN_PATH)
    
    # 2. Definição
    modelos = definir_modelos()

    # 3. Treino e Persistência
    for nome, modelo in modelos.items():
        print(f"\n--- Treinando {nome} ---")
        modelo.fit(X_train, y_train)
        salvar_modelo(modelo, nome)

    print("\nTodos os modelos foram treinados e salvos com sucesso!")

if __name__ == "__main__":
    executar_pipeline_treinamento()


--- Treinando RandomForest ---
Modelo RandomForest salvo em: ../src/models/RandomForest.pkl

--- Treinando XGBoost ---


/home/MarceloMelo/Documentos/smartflow-health-analytics/venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [20:34:43] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Modelo XGBoost salvo em: ../src/models/XGBoost.pkl

--- Treinando LogisticRegression ---
Modelo LogisticRegression salvo em: ../src/models/LogisticRegression.pkl

--- Treinando SVM ---
Modelo SVM salvo em: ../src/models/SVM.pkl

Todos os modelos foram treinados e salvos com sucesso!


5. MODELAGEM

A etapa de modelagem representa o núcleo de inteligência do projeto, onde os dados processados e escalonados são utilizados para treinar algoritmos capazes de realizar a predição do nível de triagem. Esta fase foi desenhada como um experimento comparativo, submetendo o dataset a diferentes paradigmas de aprendizado para identificar qual arquitetura oferece a melhor capacidade preditiva e generalização para o cenário hospitalar.

5.1. Seleção e Fundamentação dos Modelos
Foram selecionados quatro algoritmos distintos, cada um representando uma abordagem matemática diferente para o problema de classificação:

Random Forest Classifier: Um modelo de Ensemble Learning baseado em múltiplas árvores de decisão. Foi escolhido por sua robustez e baixa tendência ao overfitting. Em saúde, sua escolha é justificada pela capacidade de lidar com variáveis de diferentes naturezas (sinais vitais e histórico clínico) e capturar interações não lineares complexas.

XGBoost (Extreme Gradient Boosting): Um algoritmo de Gradient Boosting de alta performance. Diferente da Random Forest, ele foca em corrigir sequencialmente os erros das árvores anteriores. Foi escolhido pela sua extrema eficiência em detectar padrões sutis em dados tabulares, o que é vital para separar classes de urgência com limites numéricos muito próximos.

Regressão Logística: Implementada como o baseline linear do projeto. Sua escolha fundamenta-se na necessidade de estabelecer uma base de comparação simples e altamente interpretável, permitindo validar se a complexidade dos modelos de árvores ou kernels é realmente necessária para o problema.

SVM (Support Vector Machine) com Kernel RBF: Escolhido por sua excelência em encontrar hiperplanos de separação em espaços de alta dimensão. O uso do kernel RBF permite que o modelo crie fronteiras de decisão flexíveis e não lineares, sendo ideal para classificar sinais vitais onde a separação entre "estável" e "crítico" pode não ser uma linha reta.

5.2. Configuração e Justificativa dos Hiperparâmetros
Cada modelo foi configurado meticulosamente para equilibrar o desempenho e a estabilidade estatística:

Random Forest

n_estimators=100 e max_depth=10: Configurou-se um conjunto de 100 árvores com profundidade limitada a 10 níveis. Esta profundidade controlada é uma decisão estratégica para garantir que o modelo aprenda padrões clínicos globais em vez de memorizar casos específicos do treino, promovendo maior robustez.

class_weight='balanced': Essencial para lidar com o desbalanceamento, este parâmetro atribui um peso maior aos erros cometidos nas classes minoritárias (Níveis 2 e 3), garantindo que a IA não ignore pacientes graves em favor da acurácia global.

XGBoost

eval_metric='mlogloss': Utilizou-se a perda logarítmica multiclasse como critério de avaliação interna. Esta métrica penaliza previsões confiantes, mas erradas, forçando o modelo a ser mais preciso nas probabilidades atribuídas a cada nível de triagem.

use_label_encoder=False: Aplicado para manter a consistência com o pré-processamento manual de rótulos já realizado, evitando conflitos de codificação entre bibliotecas.

Regressão Logística

max_iter=1000: O limite de iterações do solver foi expandido para garantir a convergência total do algoritmo. Como o problema envolve múltiplas classes e dados escalonados pelo RobustScaler, um limite maior assegura que o modelo encontre o ponto de erro mínimo sem interrupções prematuras.

class_weight='balanced': Aplicado para dar importância proporcional aos casos críticos dentro da função de perda linear.

SVM (Support Vector Machine)

kernel='rbf' e probability=True: O kernel RBF foi configurado para permitir a separação não linear. A habilitação de probabilidades é fundamental para que o modelo não retorne apenas uma classe rígida, mas também o nível de confiança na predição, o que agrega valor na tomada de decisão médica.

class_weight='balanced': Garante que o hiperplano de separação seja posicionado de forma a respeitar a importância dos casos raros (Nível 3).

5.3. Padronização e Reprodutibilidade
Todos os modelos foram inicializados com random_state=42. Esta configuração garante que todos os processos estocásticos (aleatórios) internos dos algoritmos, como a seleção de atributos na Random Forest ou a inicialização de pesos no SVM, sejam idênticos em cada execução. Isso é fundamental para a integridade científica do relatório, permitindo que os resultados comparativos sejam replicados com exatidão por outros pesquisadores.